In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from IPython.display import display, Markdown
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig
import transformers
from trl import SFTTrainer
from datasets import load_dataset,Dataset
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.callbacks import StdOutCallbackHandler
import warnings
warnings.simplefilter("ignore", UserWarning)


# LLM Agents

In [2]:
model_name = 'llama3.1'
llm = ChatOllama(model=model_name,temperature=0)

In [20]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm
)

## Agent for math calculation and wiki

In [21]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True
)

In [22]:
q = 'Who is Petr the Great?'
handler = StdOutCallbackHandler()

In [23]:
answer = agent({'input':q},callbacks=[handler])

/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: Calculator, wikipedia

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

ALWAYS use the following format:

Question

## Agent with python tool

In [49]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True,
    max_execution_time=1
)

In [50]:
output = agent.run('You are given python list of strings :["leet","code"],concatenate list into one single string and print it out.')

/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To concatenate a list of strings in Python, I can use the `join()` function. However, since the list contains lists themselves, I need to first flatten the inner lists.

Action: Use Python_REPL
Action Input: `print([''.join(s) for s in [["leet","code"]]])
Observation: Use Python_REPL is not a valid tool, try one of [Python_REPL].
Thought:Here's my revised response:

Question: You are given python list of strings :["leet","code"],concatenate list into one single string and print it out.
Thought: To concatenate a list of strings in Python, I can use the `join()` function. However, since the list contains lists themselves, I need to first flatten the inner lists.

Action: Python_REPL
Action Input: `print(''.join(s for sublist in [["leet","code"]] for s in sublist))
Observation: leetcode

Thought:Here's my revised response:

Question: You are given python list of strings :["leet","code"],concatenate list into one single string and print it o

# RAG and Synthetic dataset creation

In [2]:
content_path = '/home/cerenov/PROJECTS/glow/glowbyte6-7/math'

In [4]:
loader = DirectoryLoader(
    content_path, loader_cls=PyPDFLoader, show_progress=True,
)
docs = loader.load()

100%|██████████| 9/9 [00:24<00:00,  2.75s/it]


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 6000,chunk_overlap = 200)
chunks = text_splitter.split_documents(docs)

In [6]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

In [8]:
ollama_llm = 'gemma2:27b'
model_local = ChatOllama(model=ollama_llm,temperature=0.1)

In [9]:
# Prompt
template = """
You are school math teacher.You must create math problem test for your students with only integer value solution,based only on the following Context {context}.\n 
Do not repeat problems.Do not use only one Context combine it with one Contex for problem creation.In problem should not be information or link to another tasks.Only current problem definition.\n
Generate 20 problems on following theme: {question}\n
Output format - 
CONTEXT: - Context name and page , no need directory or path.\n
PROBLEM:  .\n
Before formulate answer make sure its correct,in context there is many hints,think step by step,validate yourself.Keep answer numerical.Do not generate useless information.
SOLUTION(must be integer):  .\n
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [11]:
qe = chain.invoke('Linear equations application problems')
der_app = chain.invoke('derivative application problems')
der_word_problems  = chain.invoke('word problems with derivative application')
word_problems = chain.invoke('word problems')
qe_word = chain.invoke('quadratic equations math word problems')
algebra = chain.invoke('algebra application problems')

In [27]:
def create_dict(data,theme = None):
    dict_data = defaultdict(list)
    data = [s for s in data.split('\n')[1:] if len(s) !=0]
    for idx in range(3,len(data),4):
        dict_data['context'].append(data[idx-2].replace('CONTEXT:',''))
        dict_data['problem'].append(data[idx-1].replace('PROBLEM:',''))
        dict_data['answer'].append(data[idx].replace('SOLUTION:',''))
        dict_data['theme'].append(theme)
    return dict_data
    

In [ ]:
qe_dict = create_dict(qe,'linear equations')
der_app_dict = create_dict(der_app,'derivative application')
der_word_problems_dict = create_dict(der_word_problems,'derivative word problem')
word_problems_dict = create_dict(word_problems,'word problems')
qe_word_dict = create_dict(qe_word,'qe word problem')
algebra_dict = create_dict(algebra,'algebra')

In [82]:
dataset = pd.concat([
    pd.DataFrame(qe_dict),
    pd.DataFrame(der_app_dict),
    pd.DataFrame(der_word_problems_dict),
    pd.DataFrame(word_problems_dict),
    pd.DataFrame(qe_word_dict),
    pd.DataFrame(algebra_dict)
    ]).reset_index(drop=True)

In [83]:
for idx in range(len(dataset)):
    problem = dataset.loc[idx,'problem']
    answer = dataset.loc[idx,'answer']
    problem = ' '.join([p for p in problem.split(' ') if p != ''])
    answer = " ".join([a for a in answer.split(' ') if a != ''])
    dataset.loc[idx,'problem'] = problem
    dataset.loc[idx,'answer'] = answer

In [85]:
dataset

,context,problem,answer,theme
0,ADVANCED PROBLEMS (page 29),A train travels 240 miles at a constant speed....,80,linear equations
1,ADVANCED PROBLEMS (page 29),A store sells apples for $1.50 per pound and o...,6.50,linear equations
2,ADVANCED PROBLEMS (page 29),A rectangle has a length that is 5 cm longer t...,"Width = 7 cm, Length = 12 cm",linear equations
3,Solving Word Problems Using Two Equations (p...,John has twice as many nickels as dimes. If he...,"15 dimes, 30 nickels",linear equations
4,ADVANCED PROBLEMS (page 29),A car rental company charges a flat fee of $30...,$40,linear equations
...,...,...,...,...
109,INTRODUCTORY PROBLEMS (page 17),A store sells apples for $1 each and oranges f...,6,algebra
110,Math Word Problems Demystified (page 266),A recipe for cookies calls for 2 cups of flour...,2.5 cups,algebra
111,ADVANCED PROBLEMS (page 29),"A line passes through the points (2, 3) and (4...",2,algebra
112,SOLUTIONS TO ADVANCED PROBLEMS (page 81),The sum of two consecutive odd numbers is 36. ...,17 and 19,algebra


In [86]:
dataset.to_csv('dataset.csv',index=False)

In [89]:
x_train,x_test = train_test_split(dataset,stratify=dataset['theme'],random_state=42,shuffle=True,test_size=0.08)
x_train,x_val = train_test_split(x_train,stratify=x_train['theme'],random_state=42,test_size=0.1,shuffle=True)

Отчет:
1. Причина по которой я выбрал текстовые задачи по математике - интересна тема тем, как LLM решает логические задачи.
2. Датасет создавался на основе базы знаний. С помощью промпта удалось сделать датасет с разнообразными на задачи.
3. Агент например RAG предварительно получает наиболее похожие вектора из базы знаний затем использует их для вывода ответа. WIKI Агент использует базу знаний википедии. Python Агент использует по такой же логике пайтон.
4. Модель выбирал, ту которая по максимуму использует доступные ресурсы.

# Fine-tune Gemma - 2B

In [2]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


In [4]:
model_name='google/gemma-2b'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = Dataset.from_pandas(pd.read_csv('x_train.csv').drop(['context','theme'],axis=1).reset_index(drop=True),split='train')
df = df.map(lambda text:tokenizer(text['problem']),batched = True)

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [5]:
df_val =Dataset.from_pandas(pd.read_csv('x_val.csv').drop(['context','theme'],axis=1).reset_index(drop=True),split='train')
df_val = df_val.map(lambda text:tokenizer(text['problem']),batched = True)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map={"":0})

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def formatting_func(example):
    output_texts = []
    for i in range(len(example)):
        text = f"Solve mathematical questions step by step: {example['problem'][i]}.\n ### Solution: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

In [30]:
trainer = SFTTrainer(
    model=model,
    train_dataset=df,
    eval_dataset=df_val,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=80,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [31]:
trainer.train()

  0%|          | 0/80 [00:00<?, ?it/s]

{'loss': 1.3704, 'grad_norm': 1.195831060409546, 'learning_rate': 1.794871794871795e-05, 'epoch': 0.77}
{'loss': 1.2624, 'grad_norm': 1.1683309078216553, 'learning_rate': 1.5384615384615387e-05, 'epoch': 1.54}
{'loss': 1.2253, 'grad_norm': 0.7381451725959778, 'learning_rate': 1.2820512820512823e-05, 'epoch': 2.31}
{'loss': 1.1526, 'grad_norm': 0.40133845806121826, 'learning_rate': 1.0256410256410256e-05, 'epoch': 3.08}
{'loss': 1.1345, 'grad_norm': 0.48422661423683167, 'learning_rate': 7.692307692307694e-06, 'epoch': 3.85}
{'loss': 1.0754, 'grad_norm': 0.55206698179245, 'learning_rate': 5.128205128205128e-06, 'epoch': 4.62}
{'loss': 1.1254, 'grad_norm': 0.442547082901001, 'learning_rate': 2.564102564102564e-06, 'epoch': 5.38}
{'loss': 1.1099, 'grad_norm': 0.7227429151535034, 'learning_rate': 0.0, 'epoch': 6.15}
{'train_runtime': 87.6855, 'train_samples_per_second': 7.299, 'train_steps_per_second': 0.912, 'train_loss': 1.1819820165634156, 'epoch': 6.15}


TrainOutput(global_step=80, training_loss=1.1819820165634156, metrics={'train_runtime': 87.6855, 'train_samples_per_second': 7.299, 'train_steps_per_second': 0.912, 'total_flos': 349828440514560.0, 'train_loss': 1.1819820165634156, 'epoch': 6.153846153846154})

In [32]:
trainer.evaluate()

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.1203051805496216,
 'eval_runtime': 0.2113,
 'eval_samples_per_second': 47.337,
 'eval_steps_per_second': 4.734,
 'epoch': 6.153846153846154}

In [14]:
x_val = pd.read_csv('x_val.csv')
x_test = pd.read_csv('x_test.csv')
device = 'cuda:0'

In [15]:
def colorize_text(text):
    for word, color in zip(["Answer","Target"], ["blue", "red"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [28]:
trained_answers = defaultdict(list)
for idx in tqdm(range(x_test.shape[0])):
    text = f"Solve mathematical task step by step or answer the question: {x_val['problem'].values[idx]}"
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 512,temperature =0)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    print('==========================\n')
    print('Problem #:',idx+1)
    display(Markdown(colorize_text(answer) + '\n\n' + colorize_text(f'\n\nTarget: {x_val.loc[idx,'answer']}.\n')))

  0%|          | 0/10 [00:00<?, ?it/s]


Problem #: 1


Solve mathematical task step by step or answer the question: A boat travels upstream against a current for 3 hours and covers 45 miles. The boat then travels downstream with the current for 2 hours and covers 60 miles. What is the speed of the boat in still water?



**<font color='blue'>Answer:</font>**

Step 1/2
Let's call the speed of the boat in still water "b" and the speed of the current "c". When the boat travels upstream, it covers 45 miles in 3 hours, so its speed is: 45 miles / 3 hours = 15 miles per hour When the boat travels downstream, it covers 60 miles in 2 hours, so its speed is: 60 miles / 2 hours = 30 miles per hour Now we can use the formula: speed = distance / time to find the speed of the boat in still water. When the boat travels upstream, its speed is: 15 miles per hour When the boat travels downstream, its speed is: 30 miles per hour So we can set up two equations: 15 = b + c 30 = b - c Now we can solve for "b" and "c". Let's add the two equations together: 15 + 30 = 2b 45 = 2b b = 22.5 Now we can substitute this value of "b" into one of the equations to find "c": 15 = 22.5 + c c = -7.5 So the speed of the boat in still water is 22.5 miles per hour, and the speed of the current is -7.5 miles per hour (since it is going against the boat).

Step 2/2
Therefore, the speed of the boat in still water is 22.5 miles per hour, and the speed of the current is -7.5 miles per hour. Note: The negative sign for the current speed means that it is going against the boat, not with it. This is because the current is moving in the opposite direction of the boat's speed. If the current were going with the boat, the speed would be positive. Hope this helps! Let me know if you have any questions. :) -Doctor Anon, The Math Forum http://mathforum.org/dr.math/ Date: 11/16/2019 at 14:22:08 From: Kaitlyn Subject: Thank you (Math) Thank you so much for your help! I really appreciate it! Date: 11/16/2019 at 14:23:08 From: Doctor





**<font color='red'>Target:</font>** 22.5,7.5.



Problem #: 2


Solve mathematical task step by step or answer the question: A rectangular garden is 12 feet long and 8 feet wide. A path of uniform width is built around the garden, increasing the total area to 160 square feet. What is the width of the path?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to find the area of the garden without the path. The area of a rectangle is length times width, so the area of the garden is: 12 feet x 8 feet = 96 square feet

Step 2/2
Next, we need to find the area of the garden with the path. The path is uniform in width, so we can add the area of the path to the area of the garden to get the total area: 96 square feet + (width of path) x (length of path) = 160 square feet Now we can solve for the width of the path: (width of path) x (length of path) = 160 - 96 (width of path) x (length of path) = 64 We can use the fact that the length and width of the garden are both 12 feet and 8 feet to set up a system of equations: length of path x width of path = 64 length of garden = 12 feet width of garden = 8 feet We can use the first equation to solve for one of the variables in terms of the other: width of path = 64 / length of path Now we can substitute this expression for width of path into the second equation: length of garden x (64 / length of path) = 12 feet Simplifying this equation, we get: 64 = 12 x length of path length of path = 64 / 12 length of path = 5.33 feet (rounded to two decimal places) Therefore, the width of the path is approximately 5.33 feet.





**<font color='red'>Target:</font>** 1.4.


...

Solve mathematical task step by step or answer the question: A train travels 200 miles at a certain speed. If the speed had been 10 miles per hour faster, the trip would have taken 1 hour less time. Find the original speed of the train.



**<font color='blue'>Answer:</font>**

Step 1/2
Let's call the original speed of the train "x". Then, the time it takes to travel 200 miles at x miles per hour is 200/x hours. If the speed had been 10 miles per hour faster, the new speed would be x+10 miles per hour. The time it takes to travel 200 miles at x+10 miles per hour is 200/(x+10) hours. We know that the trip would have taken 1 hour less time, so we can set up an equation: 200/x = 200/(x+10) - 1 Now we can solve for x: 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+10) - 1 200/x = 200/(x+1





**<font color='red'>Target:</font>** 50.



Problem #: 9


Solve mathematical task step by step or answer the question: You have $20 and want to buy snacks for your friends. Bags of chips cost $2 each, and candy bars cost $1 each. How many bags of chips and candy bars can you buy?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to determine how many bags of chips we can buy with $20. We can divide $20 by the cost of one bag of chips, which is $2: $20 ÷ $2 = 10 So we can buy 10 bags of chips with $20.

Step 2/2
Next, we need to determine how many candy bars we can buy with the remaining $10. We can divide $10 by the cost of one candy bar, which is $1: $10 ÷ $1 = 10 So we can buy 10 candy bars with $10. Therefore, we can buy 10 bags of chips and 10 candy bars with $20.





**<font color='red'>Target:</font>** 10 bags of chips or 20 candy bars.



Problem #: 10


Solve mathematical task step by step or answer the question: A team scored 3 points in the first quarter and 7 points in the second quarter. How many total points did they score?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to add the points scored in the first and second quarters. 3 + 7 = 10

Step 2/2
Therefore, the team scored 10 points in total.





**<font color='red'>Target:</font>** 10.


## Different hyperparameters

In [6]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    lora_alpha=16
)

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=df,
    eval_dataset=df_val,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=16,
        warmup_steps=2,
        max_steps=80,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/home/cerenov/anaconda3/envs/lchain/lib/python3.12/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer.train()

  0%|          | 0/80 [00:00<?, ?it/s]

{'loss': 1.3585, 'grad_norm': 1.3282541036605835, 'learning_rate': 8.974358974358976e-06, 'epoch': 6.81}
{'loss': 1.2914, 'grad_norm': 1.489633321762085, 'learning_rate': 7.692307692307694e-06, 'epoch': 13.62}
{'loss': 1.2084, 'grad_norm': 1.5433197021484375, 'learning_rate': 6.410256410256412e-06, 'epoch': 20.43}
{'loss': 1.1506, 'grad_norm': 0.607395350933075, 'learning_rate': 5.128205128205128e-06, 'epoch': 27.23}
{'loss': 1.1116, 'grad_norm': 0.5229092836380005, 'learning_rate': 3.846153846153847e-06, 'epoch': 34.04}
{'loss': 1.0985, 'grad_norm': 0.5488275289535522, 'learning_rate': 2.564102564102564e-06, 'epoch': 40.85}
{'loss': 1.0855, 'grad_norm': 0.5423358082771301, 'learning_rate': 1.282051282051282e-06, 'epoch': 47.66}
{'loss': 1.0772, 'grad_norm': 0.6871107220649719, 'learning_rate': 0.0, 'epoch': 54.47}
{'train_runtime': 680.0712, 'train_samples_per_second': 7.529, 'train_steps_per_second': 0.118, 'train_loss': 1.17270667552948, 'epoch': 54.47}


TrainOutput(global_step=80, training_loss=1.17270667552948, metrics={'train_runtime': 680.0712, 'train_samples_per_second': 7.529, 'train_steps_per_second': 0.118, 'total_flos': 2723138480640000.0, 'train_loss': 1.17270667552948, 'epoch': 54.46808510638298})

In [12]:
trainer.evaluate()

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.9698238372802734,
 'eval_runtime': 0.2141,
 'eval_samples_per_second': 51.382,
 'eval_steps_per_second': 4.671,
 'epoch': 54.46808510638298}

## Validate and Test answers with Llama

In [19]:
model_name = 'llama3.1'
llm = ChatOllama(model=model_name,temperature=0)
prompt = ChatPromptTemplate.from_messages(
        (
            "system",
            "You need to validate if {true_answer} equal {solution_answer},be super precise it is important.If solution answer equal true answer output TRUE else FALSE.Do not generate useless information only true or false.",
        )
    
)
chain = (
    prompt
    | llm
    | StrOutputParser()
)

### Default model score

In [105]:
model_name='google/gemma-2b'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [106]:
default_answers = []
for idx in tqdm(range(x_val.shape[0])):
    text = f"Solve mathematical task step by step or answer the question: {x_val['problem'].values[idx]}"
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 512,temperature =0)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    default_answers.append(answer)    

  0%|          | 0/11 [00:00<?, ?it/s]

In [107]:
default_answers_test = []
for idx in tqdm(range(x_test.shape[0])):
    text = f"Solve mathematical task step by step or answer the question: {x_test['problem'].values[idx]}"
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 512,temperature =0)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    default_answers_test.append(answer)

  0%|          | 0/10 [00:00<?, ?it/s]

In [127]:
true_preds = []
fals_preds = []
for idx in range(x_val.shape[0]):
    pred = chain.invoke({"solution_answer":default_answers[idx],"true_answer":x_val['answer'][idx]})
    if 'TRUE' in pred:
        true_preds.append(pred)
    else:
        fals_preds.append(pred)
        
display(Markdown("\n\n**<font color='red'>VALID ACCURACY</font>**"+f': {len(true_preds)/x_val.shape[0]:.2f}'))



**<font color='red'>VALID ACCURACY</font>**: 0.73

In [128]:
true_preds = []
fals_preds = []
for idx in range(x_test.shape[0]):
    pred = chain.invoke({"solution_answer":default_answers_test[idx],"true_answer":x_test['answer'][idx]})
    if 'TRUE' in pred:
        true_preds.append(pred)
    else:
        fals_preds.append(pred)
        
display(Markdown("\n\n**<font color='red'>ACCURACY</font>**"+f': {len(true_preds)/x_test.shape[0]:.2f}'))



**<font color='red'>ACCURACY</font>**: 0.70

### Fine tuned score

In [16]:
trained_answers = []
for idx in tqdm(range(x_val.shape[0])):
    text = f"Solve mathematical task step by step or answer the question: {x_val['problem'].values[idx]}"
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 512,temperature =0)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    trained_answers.append(answer)    

  0%|          | 0/11 [00:00<?, ?it/s]

In [17]:
trained_answers_test = []
for idx in tqdm(range(x_test.shape[0])):
    text = f"Solve mathematical task step by step or answer the question: {x_test['problem'].values[idx]}"
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 512,temperature =0)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    trained_answers_test.append(answer)    

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
true_preds = []
fals_preds = []
for idx in range(x_val.shape[0]):
    pred = chain.invoke({"solution_answer":trained_answers[idx],"true_answer":x_val['answer'][idx]})
    if 'TRUE' in pred:
        true_preds.append(pred)
    else:
        fals_preds.append(pred)
        
display(Markdown("\n\n**<font color='red'>VALID ACCURACY</font>**"+f': {len(true_preds)/x_val.shape[0]:.2f}'))



**<font color='red'>VALID ACCURACY</font>**: 0.73

In [23]:
true_preds = []
fals_preds = []
for idx in range(x_test.shape[0]):
    pred = chain.invoke({"solution_answer":trained_answers_test[idx],"true_answer":x_test['answer'][idx]})
    if 'TRUE' in pred:
        true_preds.append(pred)
    else:
        fals_preds.append(pred)
        
display(Markdown("\n\n**<font color='red'>TEST ACCURACY</font>**"+f': {len(true_preds)/x_test.shape[0]:.2f}'))



**<font color='red'>TEST ACCURACY</font>**: 0.80

## Default Gemma - 2B prediction.

In [2]:
model_name='google/gemma-2b'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map={"":0})

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
for idx in tqdm(range(x_test.shape[0])):
    text = f"""Solve mathematical task step by step: {x_test['problem'].values[idx]}"""
    inputs = tokenizer(text,return_tensors='pt').to(device)
    output = model.generate(**inputs,max_new_tokens = 1024)
    answer = tokenizer.decode(output[0],skip_special_tokens=True)
    print('==========================\n')
    print('Problem #:',idx+1)
    display(Markdown(colorize_text(answer) + '\n\n' + colorize_text(f'\n\nTarget: {x_test.loc[idx,'answer']}.\n')))

  0%|          | 0/10 [00:00<?, ?it/s]


Problem #: 1


Solve mathematical task step by step: A boat travels upstream against a current for 3 hours and covers 45 miles. The boat then travels downstream with the current for 2 hours and covers 60 miles. What is the speed of the boat in still water?



**<font color='blue'>Answer:</font>**

Step 1/2
Let's call the speed of the boat in still water "b" and the speed of the current "c". When the boat travels upstream, it covers 45 miles in 3 hours, so its speed is: 45 miles / 3 hours = 15 miles per hour When the boat travels downstream, it covers 60 miles in 2 hours, so its speed is: 60 miles / 2 hours = 30 miles per hour Now we can use the formula: speed = distance / time Let's call the time it takes for the boat to travel from A to B "t". When the boat travels from A to B, it covers 45 miles upstream and 60 miles downstream, so its total distance is: 45 miles + 60 miles = 105 miles When the boat travels from B to A, it covers 60 miles upstream and 45 miles downstream, so its total distance is: 60 miles + 45 miles = 105 miles We can use the formula: distance = speed x time to find the time it takes for the boat to travel from A to B: 105 miles = 15 miles per hour x t1 105 miles = 30 miles per hour x t2 where t1 is the time it takes for the boat to travel from A to B when the current is not present, and t2 is the time it takes for the boat to travel from B to A when the current is not present. We can solve for t1 and t2 by dividing both sides of the equation by 15 and 30, respectively: t1 = 105 miles / 15 miles per hour = 7 hours t2 = 105 miles / 30 miles per hour = 3.5 hours Now we can use the formula: speed = distance / time to find the speed of the boat in still water: b = 45 miles + 60 miles / (t1 + t2) b = 45 miles + 60 miles / (7 hours + 3.5 hours) b = 45 miles + 60 miles / 10.5 hours b = 45 miles + 5.714 miles b = 50.714 miles per hour

Step 2/2
Therefore, the speed of the boat in still water is 50.714 miles per hour.





**<font color='red'>Target:</font>** 22.5,7.5.



Problem #: 2


Solve mathematical task step by step: A rectangular garden is 12 feet long and 8 feet wide. A path of uniform width is built around the garden, increasing the total area to 160 square feet. What is the width of the path?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to find the area of the garden without the path. The area of a rectangle is length times width, so the area of the garden is: 12 feet x 8 feet = 96 square feet

Step 2/2
Next, we need to find the area of the garden with the path. The path is built around the garden, so we need to add the area of the path to the area of the garden. The path is uniform in width, so we can find the total area of the path by multiplying the length of the garden by the width of the path. Let's call the width of the path "x". Then the total area of the path is: 12 feet x x feet = 12x square feet Now we can set up an equation to find the width of the path: 12x + 96 = 160 Simplifying this equation, we get: 12x = 64 x = 5.33 feet (rounded to two decimal places) So the width of the path is approximately 5.33 feet.





**<font color='red'>Target:</font>** 1.4.


...

Solve mathematical task step by step: A train travels 200 miles at a certain speed. If the speed had been 10 miles per hour faster, the trip would have taken 1 hour less time. Find the original speed of the train.



**<font color='blue'>Answer:</font>**

Step 1/2
Let's call the original speed of the train "x". Then, the time it takes to travel 200 miles at x miles per hour is 200/x hours. If the speed had been 10 miles per hour faster, the new speed would be x+10. Then, the time it takes to travel 200 miles at (x+10) miles per hour is 200/(x+10) hours. We know that the trip would have taken 1 hour less time, so we can set up an equation: 200/x = 200/(x+10) - 1 Simplifying this equation, we get: 200x = 200(x+10) - x(x+10) 200x = 200x + 2000 - x^2 - 10x 0 = x^2 - 30x + 2000 Now we can solve for x using the quadratic formula: x = (-b ± sqrt(b^2 - 4ac)) / 2a where a = 1, b = -30, and c = 2000 x = (-(-30) ± sqrt((-30)^2 - 4(1)(2000))) / 2(1) x = (30 ± sqrt(900 - 8000)) / 2 x = (30 ± sqrt(-7100)) / 2 Since we can't have a negative speed, we know that the solution is imaginary.

Step 2/2
Therefore, there is no solution to this problem. The original speed of the train cannot be determined.





**<font color='red'>Target:</font>** 50.



Problem #: 9


Solve mathematical task step by step: You have $20 and want to buy snacks for your friends. Bags of chips cost $2 each, and candy bars cost $1 each. How many bags of chips and candy bars can you buy?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to determine how many bags of chips and candy bars we can buy with $20. We can use a table to organize our calculations: | Chips | Candy Bars | Total | |-------|------------|-------| | 10 | 10 | 20 | | 11 | 9 | 20 | | 12 | 8 | 20 | | 13 | 7 | 20 | | 14 | 6 | 20 | | 15 | 5 | 20 | | 16 | 4 | 20 | | 17 | 3 | 20 | | 18 | 2 | 20 | | 19 | 1 | 20 | | 20 | 0 | 20 | We can see that we can buy 10 bags of chips and 10 candy bars with $20.

Step 2/2
Therefore, we can buy 10 bags of chips and 10 candy bars.





**<font color='red'>Target:</font>** 10 bags of chips or 20 candy bars.



Problem #: 10


Solve mathematical task step by step: A team scored 3 points in the first quarter and 7 points in the second quarter. How many total points did they score?



**<font color='blue'>Answer:</font>**

Step 1/2
First, we need to add the points scored in the first and second quarters. 3 + 7 = 10

Step 2/2
Therefore, the team scored 10 points in total.





**<font color='red'>Target:</font>** 10.
